In [1]:
from datasets import load_dataset

python_dataset = load_dataset("blindsubmissions/GH_text2code", split="python_gh")

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/73 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating go_gh split:   0%|          | 0/15000078 [00:00<?, ? examples/s]

Generating java_gh split:   0%|          | 0/15000014 [00:00<?, ? examples/s]

Generating javascript_gh split:   0%|          | 0/2000040 [00:00<?, ? examples/s]

Generating python_gh split:   0%|          | 0/15000002 [00:00<?, ? examples/s]

In [2]:
python_dataset

Dataset({
    features: ['identifier', 'parameters', 'docstring', 'docstring_summary', 'function', 'function_tokens', 'start_point', 'end_point', 'language', 'docstring_language', 'docstring_language_predictions', 'is_langid_reliable'],
    num_rows: 15000002
})

In [6]:
print(python_dataset[0]['function'])

async def async_setup_entry(hass, entry, async_add_entities):
    """Set up SimpliSafe locks based on a config entry."""
    simplisafe = hass.data[DOMAIN][DATA_CLIENT][entry.entry_id]
    async_add_entities(
        [
            SimpliSafeLock(simplisafe, system, lock)
            for system in simplisafe.systems.values()
            for lock in system.locks.values()
        ]
    )


In [7]:
import ast

def find_undefined_variables(code):
    undefined_variables = set()

    class VariableVisitor(ast.NodeVisitor):
        def visit_Name(self, node):
            if isinstance(node.ctx, ast.Store):
                # If the node is a variable being assigned to, add it to the defined variables set
                defined_variables.add(node.id)
            elif isinstance(node.ctx, ast.Load) and node.id not in defined_variables:
                # If the node is a variable being loaded and it's not in the defined variables set, it's undefined
                undefined_variables.add(node.id)

    tree = ast.parse(code)
    defined_variables = set()
    visitor = VariableVisitor()
    visitor.visit(tree)

    return undefined_variables

In [8]:
code = """
x = 10
print(x + y)
"""
undefined_vars = find_undefined_variables(code)
if undefined_vars:
    print("Undefined variables:", undefined_vars)
else:
    print("No undefined variables found.")

Undefined variables: {'y', 'print'}


In [22]:
import subprocess
from io import StringIO

In [36]:
import pylint.lint

def check_for_undefined_variables(code):
    # Run pylint on the code string
    (stdout, _) = pylint.lint.Run(["--disable=all", "--enable=undefined-variable", "--output-format=json",], exit=False).linter.check(code)

    # Extract the undefined variable messages from the output
    undefined_vars = []
    for message in stdout:
        if message.get('type') == 'undefined-variable':
            undefined_vars.append(message)

    return undefined_vars

In [37]:
# Your dataset of Python source code strings
python_sources = [
    "x = 42\nprint(x + y)",
    "for i in range(5):\n    print(i)\nprint(j)"
    # Add more Python source code strings as needed
]

for idx, source_code in enumerate(python_sources, start=1):
    print(f"Checking source code {idx}")
    undefined_vars = check_for_undefined_variables(source_code)
    if undefined_vars:
        print("Undefined variables found:")
        for undefined_var in undefined_vars:
            print(f"    - {undefined_var['message']}")

Checking source code 1
No files to lint: exiting.


SystemExit: 32

In [39]:
import ast

def find_undefined_variables(code):
    undefined_vars = set()

    class UndefinedVariableFinder(ast.NodeVisitor):
        def __init__(self):
            self.variables = set()

        def visit_Name(self, node):
            self.variables.add(node.id)

        def visit_FunctionDef(self, node):
            # Exclude variables defined inside functions
            self.variables -= set(arg.arg for arg in node.args.args)
            self.generic_visit(node)

    try:
        tree = ast.parse(code)
    except SyntaxError:
        # Handle syntax errors in the code
        return {"SyntaxError"}

    finder = UndefinedVariableFinder()
    finder.visit(tree)

    # Dummy code execution to detect undefined variables
    try:
        exec(code, {}, {})
    except NameError as e:
        undefined_vars.add(e.args[0])

    # Subtract defined variables from undefined variables
    undefined_vars -= finder.variables
    return undefined_vars

# # Example usage
# source_code = "a = 5\nb = a + c"
# undefined_variables = find_undefined_variables(source_code)
# print("Undefined variables:", undefined_variables)

In [43]:
code = python_dataset[0]['function']
undefined_variables = find_undefined_variables(code)

In [44]:
undefined_variables

set()

In [45]:
print(code)

async def async_setup_entry(hass, entry, async_add_entities):
    """Set up SimpliSafe locks based on a config entry."""
    simplisafe = hass.data[DOMAIN][DATA_CLIENT][entry.entry_id]
    async_add_entities(
        [
            SimpliSafeLock(simplisafe, system, lock)
            for system in simplisafe.systems.values()
            for lock in system.locks.values()
        ]
    )


In [46]:
def f():
    return a

In [60]:
import pylint.lint
from pylint.reporters import CollectingReporter

def check_undefined_variables(file_path):
    # Define options for pylint
    pylint_opts = [ '--disable=all', '--enable=undefined-variable']

    reporter = CollectingReporter()
    
    # Run pylint on the specified file
    pylint_output = pylint.lint.Run([file_path] + pylint_opts, exit=False, reporter=reporter)

    # print(reporter.messages)
    
    # Check if undefined variables were found
    undefined_variables_found = any(message.symbol == 'undefined-variable' for message in reporter.messages)

    return undefined_variables_found

# if __name__ == "__main__":
#     file_path = 'test.py'  # Change this to your file path
#     undefined_variables_found = check_undefined_variables(file_path)
#     if undefined_variables_found:
#         print("Undefined variables found in the code.")
#     else:
#         print("No undefined variables found in the code.")


Undefined variables found in the code.


In [ ]:
from tqdm import tqdm

undef_var_funcs = []

for i, row in enumerate(tqdm(python_dataset)):
    code = row['function']
    with open('test.py', 'w') as f:
        f.write(source_code)
    if check_undefined_variables('test.py'):
        undef_var_funcs.append(i)

In [68]:
undef_var_funcs

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [69]:
max(undef_var_funcs)

4182

In [70]:
for i in range(4182):
    if not i in undef_var_funcs:
        print(f"{i} has all defined functions")